# Nettoyage — Mubawab Location (ALL)  
Ce notebook nettoie le fichier JSON brut (scraping Selenium) et produit des fichiers propres avec **les mêmes variables** :

**id, ville, prix, surface, quartier, type_bien, nb_chambres, nb_salle_de_bain, url_annonce**

Sorties :
- `../data/clean_data/mubawab_location_all_clean.csv`
- `../data/clean_data/mubawab_location_all_clean.json`


In [21]:
# =========================
# 1) Imports & chemins
# =========================
import json, re, os
from pathlib import Path
import pandas as pd

# Ajuste ce chemin 
RAW_DIR   = Path("..") / "data" / "raw"
CLEAN_DIR = Path("..") / "data" / "clean_data"
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

RAW_JSON_MAIN   = RAW_DIR / "annonces_location_all.json"
RAW_JSON_BACKUP = RAW_DIR / "annonces_location_all_backup.json"

RAW_PATH = RAW_JSON_MAIN if RAW_JSON_MAIN.exists() else RAW_JSON_BACKUP
print("RAW_PATH =", RAW_PATH)
assert RAW_PATH.exists(), "❌ Aucun fichier JSON brut trouvé dans data/raw/"


RAW_PATH = ..\data\raw\annonces_location_all.json


In [22]:
# =========================
# 2) Charger le JSON -> DataFrame
# =========================
with open(RAW_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

# Format attendu: dict {"annonce_1": {...}, ...}
rows = []
for k, v in raw.items():
    if isinstance(v, dict):
        rows.append(v)

df = pd.DataFrame(rows)
print("Lignes:", len(df))
df.head(3)


Lignes: 4087


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bain,date_annonce,url
0,1,Casablanca,5 800 DH,130 m²,Val Fleury à Casablanca,Appartement,3 Chambres,2 Salles de bains,None,https://www.mubawab.ma/fr/a/8279130/appartemen...
1,2,Casablanca,11 500 DH,87 m²,Maârif à Casablanca,Bureau,None,1 Salle de bain,None,https://www.mubawab.ma/fr/a/8123237/appartemen...
2,3,Casablanca,7 300 DH,63 m²,Oulfa à Casablanca,Appartement,1 Chambre,1 Salle de bain,None,https://www.mubawab.ma/fr/a/8277563/studio-meu...


In [23]:
# =========================
# 3) Helpers de nettoyage
# =========================
def clean_text(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return str(x)
    x = str(x)
    x = x.replace("\xa0", " ")  # espaces insécables
    x = re.sub(r"\s+", " ", x).strip()
    return x or None

def clean_price_to_number(price_str):
    """Ex: '11 000 DH' -> 11000 ; '1.2 MDH' (rare) -> None (à adapter si besoin)"""
    s = clean_text(price_str)
    if not s:
        return None
    # garder uniquement chiffres
    digits = re.sub(r"[^0-9]", "", s)
    return int(digits) if digits else None

def normalize_city(v):
    v = clean_text(v)
    if not v:
        return None
    mapping = {
        "casa": "Casablanca",
        "casablanca": "Casablanca",
        "rabat": "Rabat",
        "marrakech": "Marrakech",
        "tanger": "Tanger",
        "tangier": "Tanger",
        "tanja": "Tanger",
    }
    key = v.lower()
    return mapping.get(key, v)

def infer_city_from_quartier(quartier):
    q = clean_text(quartier)
    if not q:
        return None
    # Cas 1: "Quartier à Ville"
    if " à " in q:
        return normalize_city(q.split(" à ")[-1].strip())
    # Cas 2: quartier == "Marrakech" / "Casablanca" etc.
    return normalize_city(q)

def infer_city_from_url(url):
    u = clean_text(url)
    if not u:
        return None
    low = u.lower()
    for token, city in [
        ("casablanca", "Casablanca"),
        ("rabat", "Rabat"),
        ("marrakech", "Marrakech"),
        ("tanger", "Tanger"),
    ]:
        if token in low:
            return city
    return None

def clean_quartier(quartier):
    q = clean_text(quartier)
    if not q:
        return None

    q = re.split(r"\s+a\s+|\s+à\s+", q, maxsplit=1)[0]
    return q.strip() or None

def clean_surface(surface):
    s = clean_text(surface)
    if not s:
        return None
    # uniformiser '100\n m²' etc.
    s = s.replace("m2", "m²")
    s = re.sub(r"\s*m²\s*", " m²", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def clean_rooms(x):
    s = clean_text(x)
    if not s:
        return None
    # Ex: "2 Chambres" -> 2
    m = re.search(r"(\d+)", s)
    return int(m.group(1)) if m else None

def clean_baths(x):
    s = clean_text(x)
    if not s:
        return None
    m = re.search(r"(\d+)", s)
    return int(m.group(1)) if m else None


In [24]:
# =========================
# 4) Nettoyer & harmoniser les colonnes
# =========================
# Harmoniser les noms entrants 
# Certains scripts utilisent 'url' au lieu de 'url_annonce'
if "url_annonce" not in df.columns and "url" in df.columns:
    df["url_annonce"] = df["url"]
elif "url_annonce" not in df.columns and "url" not in df.columns:
    df["url_annonce"] = None

# Nettoyage texte de base
for col in ["ville", "quartier", "type_bien", "surface", "prix", "url_annonce", "nb_chambres", "nb_salle_de_bain"]:
    if col in df.columns:
        df[col] = df[col].apply(clean_text)

# Surface
df["surface"] = df["surface"].apply(clean_surface)

# Prix: on garde 2 versions
df["prix_num"] = df["prix"].apply(clean_price_to_number)

# Chambres / SDB en nombres (optionnel mais utile)
df["nb_chambres_num"] = df["nb_chambres"].apply(clean_rooms)
df["nb_salle_de_bain_num"] = df["nb_salle_de_bain"].apply(clean_baths)

# Ville: compléter si manquante
df["ville"] = df["ville"].apply(normalize_city)

mask_missing_city = df["ville"].isna()
df.loc[mask_missing_city, "ville"] = df.loc[mask_missing_city, "quartier"].apply(infer_city_from_quartier)

mask_missing_city = df["ville"].isna()
df.loc[mask_missing_city, "ville"] = df.loc[mask_missing_city, "url_annonce"].apply(infer_city_from_url)

#quartier
df["quartier"] = df["quartier"].apply(clean_quartier)

# id: s'assurer integer
if "id" in df.columns:
    # certains ids peuvent être str
    df["id"] = pd.to_numeric(df["id"], errors="coerce").astype("Int64")
else:
    # si pas de id, on le reconstruit
    df["id"] = pd.RangeIndex(1, len(df)+1)

# Garder uniquement les variables demandées (exactement)
keep_cols = ["id","ville","prix","surface","quartier","type_bien","nb_chambres","nb_salle_de_bain","url_annonce"]
for c in keep_cols:
    if c not in df.columns:
        df[c] = None

df_clean = df[keep_cols].copy()

# Dédupliquer par url
df_clean = df_clean.drop_duplicates(subset=["url_annonce"]).reset_index(drop=True)

print("Après nettoyage:", len(df_clean))
df_clean.head(5)


Après nettoyage: 4036


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bain,url_annonce
0,1,Casablanca,5 800 DH,130 m²,Val Fleury,Appartement,3 Chambres,2 Salles de bains,https://www.mubawab.ma/fr/a/8279130/appartemen...
1,2,Casablanca,11 500 DH,87 m²,Maârif,Bureau,None,1 Salle de bain,https://www.mubawab.ma/fr/a/8123237/appartemen...
2,3,Casablanca,7 300 DH,63 m²,Oulfa,Appartement,1 Chambre,1 Salle de bain,https://www.mubawab.ma/fr/a/8277563/studio-meu...
3,4,Casablanca,7 500 DH,71 m²,Maârif,Appartement,2 Chambres,1 Salle de bain,https://www.mubawab.ma/fr/a/8281617/appartemen...
4,5,Casablanca,15 000 DH,73 m²,Maârif Extension,Bureau,None,1 Salle de bain,https://www.mubawab.ma/fr/a/8291727/plateau-bu...


In [25]:


# 1) Supprimer les types de bien indésirables (robuste: contient bureau/local commercial)
df_clean["type_bien"] = df_clean["type_bien"].apply(clean_text)

mask_bad_type = df_clean["type_bien"].fillna("").str.lower().str.contains(r"\bbureau\b|local\s+commercial", regex=True)
df_clean = df_clean[~mask_bad_type].reset_index(drop=True)

# 2) nb_chambres -> int (extraire le nombre depuis texte)
df_clean["nb_chambres"] = df_clean["nb_chambres"].apply(clean_rooms)

# supprimer lignes sans nb_chambres
df_clean = df_clean[df_clean["nb_chambres"].notna()].reset_index(drop=True)
df_clean["nb_chambres"] = df_clean["nb_chambres"].astype(int)

# 3) nb_salle_de_bain -> int (si manquant => 1)
df_clean["nb_salle_de_bain"] = df_clean["nb_salle_de_bain"].apply(clean_baths)
df_clean["nb_salle_de_bain"] = df_clean["nb_salle_de_bain"].fillna(1).astype(int)

# 4) prix -> float (recalculer proprement)
df_clean["prix"] = df_clean["prix"].apply(clean_price_to_number)
df_clean["prix"] = pd.to_numeric(df_clean["prix"], errors="coerce").astype(float)

# =========================
#  Remplacement prix NaN par la moyenne par ville
# =========================

# 1) Calcul de la moyenne des prix par ville (en ignorant les NaN)
moyenne_prix_ville = df_clean.groupby("ville")["prix"].mean()
# On élimine les prix irréalistes
df_clean = df_clean[df_clean["prix"].between(500, 200_000)].reset_index(drop=True)

print("Nb lignes après filtre prix:", len(df_clean))
# 2) Fonction d'imputation
def fill_prix_with_city_mean(row):
    if pd.notna(row["prix"]):
        return row["prix"]

    ville = row["ville"]
    if pd.notna(ville) and ville in moyenne_prix_ville:
        return moyenne_prix_ville[ville]

    return None  # cas très rare

# 3) Application
df_clean["prix"] = df_clean.apply(fill_prix_with_city_mean, axis=1)

# 4) Sécurité : forcer float
df_clean["prix"] = pd.to_numeric(df_clean["prix"], errors="coerce").astype(float)

print("Prix NaN après imputation :", df_clean["prix"].isna().sum())

# 5) surface -> float (extraire nombre depuis "XX m²")
def surface_to_float(x):
    s = clean_text(x)
    if not s:
        return None
    s = s.lower().replace(",", ".")
    s = s.replace("m2", "m²")

    # exemple: "100-120 m²" -> prend 100
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    return float(m.group(1)) if m else None


df_clean["surface"] = df_clean["surface"].apply(surface_to_float)
df_clean["surface"] = pd.to_numeric(df_clean["surface"], errors="coerce").astype(float)


Nb lignes après filtre prix: 3087
Prix NaN après imputation : 0


In [26]:
# Imputer surface via prix/m² (par ville + type_bien)
# =========================


# 1) Calcul du prix/m² sur les lignes valides
mask_ref = df_clean["prix"].notna() & df_clean["surface"].notna() & (df_clean["surface"] > 0)
df_ref = df_clean.loc[mask_ref, ["ville", "type_bien", "prix", "surface"]].copy()
df_ref["prix_m2"] = df_ref["prix"] / df_ref["surface"]


# Ajuste ces bornes si nécessaire selon ton dataset location
df_ref = df_ref[(df_ref["prix_m2"] >= 10) & (df_ref["prix_m2"] <= 2000)]

# 2) Prix/m² typique (médiane) par (ville, type_bien)
prix_m2_ville_type = df_ref.groupby(["ville", "type_bien"])["prix_m2"].median()

# Fallback: médiane par ville seulement
prix_m2_ville = df_ref.groupby("ville")["prix_m2"].median()

# Fallback global
prix_m2_global = df_ref["prix_m2"].median() if len(df_ref) else None

# 3) Fonction d'estimation de surface
def estimate_surface(row):
    if pd.notna(row["surface"]):  # déjà connue
        return row["surface"]
    if pd.isna(row["prix"]) or row["prix"] <= 0:
        return None

    ville = row["ville"]
    tb = row["type_bien"]

    pm2 = None
    if pd.notna(ville) and pd.notna(tb) and (ville, tb) in prix_m2_ville_type.index:
        pm2 = prix_m2_ville_type.loc[(ville, tb)]
    elif pd.notna(ville) and ville in prix_m2_ville.index:
        pm2 = prix_m2_ville.loc[ville]
    else:
        pm2 = prix_m2_global

    if pm2 is None or pd.isna(pm2) or pm2 <= 0:
        return None

    surf = row["prix"] / pm2

    # 4) garde-fous surface (à ajuster selon ton cas location)
    if surf < 10:
        return 10.0
    if surf > 1000:
        return 1000.0
    return float(surf)

# 5) Application
before = df_clean["surface"].isna().sum()
df_clean["surface"] = df_clean.apply(estimate_surface, axis=1)
after = df_clean["surface"].isna().sum()

print(f"Surface NaN avant: {before} | après estimation: {after}")


Surface NaN avant: 14 | après estimation: 0


In [27]:
print("Lignes après modifications:", len(df_clean))
print(df_clean.dtypes)
df_clean.head(3)


Lignes après modifications: 3087
id                    Int64
ville                object
prix                float64
surface             float64
quartier             object
type_bien            object
nb_chambres           int64
nb_salle_de_bain      int64
url_annonce          object
dtype: object


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bain,url_annonce
0,1,Casablanca,5800.0,130.0,Val Fleury,Appartement,3,2,https://www.mubawab.ma/fr/a/8279130/appartemen...
1,3,Casablanca,7300.0,63.0,Oulfa,Appartement,1,1,https://www.mubawab.ma/fr/a/8277563/studio-meu...
2,4,Casablanca,7500.0,71.0,Maârif,Appartement,2,1,https://www.mubawab.ma/fr/a/8281617/appartemen...


In [28]:
#  Fix final: ville manquante
# =========================
def infer_ville(row):
    v = row.get("ville")
    if pd.notna(v) and str(v).strip().lower() not in ["none", "null", ""]:
        return v

    q = str(row.get("quartier") or "").lower()
    u = str(row.get("url_annonce") or "").lower()

    if "casablanca" in q or "casablanca" in u or "casa" in q:
        return "Casablanca"
    if "marrakech" in q or "marrakech" in u:
        return "Marrakech"
    if "rabat" in q or "rabat" in u:
        return "Rabat"
    if "tanger" in q or "tanger" in u or "tangier" in q or "tangier" in u:
        return "Tanger"

    # si vraiment impossible
    return None

df_clean["ville"] = df_clean.apply(infer_ville, axis=1)

# remplace les chaînes "None" éventuelles
df_clean["ville"] = df_clean["ville"].replace({"None": None, "none": None, "NULL": None, "null": None})

print("Ville NULL après fix:", df_clean["ville"].isna().sum())
df_clean["ville"].value_counts(dropna=False).head(10)


Ville NULL après fix: 0


ville
Casablanca    1545
Marrakech      685
Rabat          508
Tanger         349
Name: count, dtype: int64

In [29]:
# =========================
# 5) Sauvegarde (CSV + JSON)
# =========================
csv_path  = CLEAN_DIR / "mubawab_location_all_clean.csv"
json_path = CLEAN_DIR / "mubawab_location_all_clean.json"

df_clean.to_csv(csv_path, index=False, encoding="utf-8-sig")

# JSON en liste d'objets
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(df_clean.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

print("✅ CSV :", csv_path)
print("✅ JSON:", json_path)


✅ CSV : ..\data\clean_data\mubawab_location_all_clean.csv
✅ JSON: ..\data\clean_data\mubawab_location_all_clean.json


In [30]:
# =========================
# 6) Contrôles rapides
# =========================
print("Ville NULL:", df_clean["ville"].isna().sum())
print("URL NULL:", df_clean["url_annonce"].isna().sum())
df_clean["ville"].value_counts(dropna=False).head(10)


Ville NULL: 0
URL NULL: 0


ville
Casablanca    1545
Marrakech      685
Rabat          508
Tanger         349
Name: count, dtype: int64